In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import numpy as np
import re
import os
from pyvirtualdisplay import Display
from selenium import webdriver

df_input = pd.read_csv('phone_info_all.csv', encoding='cp949')
df_input.iloc[0]

is_not_found = df_input['분류'] == '검색안됨'
not_found = df_input[is_not_found]
not_found.tail(5)

,제조사,모델명,코드,분류
2151,samsung,IceTouch,YP-GH1,검색안됨
2152,samsung,Galaxy Player 4.2,YP-GI1,검색안됨
2153,samsung,Galaxy 070,YP-GI2,검색안됨
2154,samsung,Galaxy Player 5.8,YP-GP1,검색안됨
2155,samsung,Galaxy Player 3.6,YP-GS1,검색안됨


In [5]:
not_found.iloc[0][2]

'10sm3tb'

In [ ]:
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "code": "",
    "country": ""
}
crawl_data = pd.DataFrame(my_dict,index=[0])
#####################################################################################
#####################################################################################
#####################################################################################

def hasxpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False

def search_by_category(df_input,crawl_data,index,option):
    try:
        url1 = 'https://www.phonemore.com/specs/?q='
        driver.get(url1+df_input.iloc[index][2])
        driver.implicitly_wait(30)

        if hasxpath('//*[@id="accessibility_content"]/div[2]/div/div[2]/div[2]/div[1]/div') == True:
            driver.implicitly_wait(50)
            a = driver.find_element_by_xpath('//*[@id="accessibility_content"]/div[2]/div/div[2]/div[2]/div[1]/div/a')
            href = a.get_attribute("href")
            driver.get(href)

            req = driver.page_source
            soup=BeautifulSoup(req, 'html.parser')
            tb_specs = soup.select("#tb_specs")
            trows = tb_specs[0].select('tr')
            for tr in trows:
                tdatas = tr.select('td')
                if tdatas[0].text == 'Country or region available':
                    country = tdatas[1].text
            my_dict = {
                "code": df_input.iloc[index][2],
                "country": country
            }

            return pd.DataFrame(my_dict,index=[0])
        
    except:
        my_dict = {
                "code": "",
                "country": ""
            }
        return pd.DataFrame(my_dict,index=[0])
        

        
# 초기화 --------------------------------------------
display = Display(visible=0, size=(800, 600))
display.start()
driver = webdriver.Chrome()
driver.implicitly_wait(3)

for i in range(1):
    crawl_data = crawl_data.append(search_by_category(not_found,crawl_data,i,"all"))


In [20]:
crawl_data

,code,country
0,,


In [17]:
crawl_data

,code,country
0,,
0,Ally,Brazil
0,EK-GC100,"(Global, International)"
0,EK-GC120,USA
0,EK-GC200,"(Global, International)"
...,...,...
0,,
0,,
0,,
0,,


In [18]:
crawl_data.to_csv('linux_country.csv',encoding='utf-8-sig')